In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('train.csv')
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
y = df["SalePrice"]
X = df.drop(columns=["SalePrice"])

In [ ]:
print(X.dtypes.value_counts())

In [ ]:
test=pd.read_csv('test.csv')
print(test.dtypes.value_counts())
colonne_df_non_in_test = set(df.columns) - set(test.columns)
print("Colonne presenti in df ma non in test:", colonne_df_non_in_test)

In [ ]:
df.isnull().count

In [ ]:

num_f = X.select_dtypes(include=[np.number]).columns
cat_features = X.select_dtypes(exclude=[np.number]).columns
X[num_f] = X[num_f].fillna(X[num_f].median())
X[cat_features] = X[cat_features].fillna("Missing")

test[num_f] = test[num_f].fillna(X[num_f].median())  
test[cat_features] = test[cat_features].fillna("Missing")
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder.fit(X[cat_features])

X_encoded = encoder.transform(X[cat_features])
test_encoded = encoder.transform(test[cat_features])

X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(), index=X.index)
test_encoded = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(), index=test.index)

X_final = X[num_f].reset_index(drop=True).join(X_encoded)
test_final = test[num_f].reset_index(drop=True).join(test_encoded)

print(X_final.shape, test_final.shape)


In [ ]:
df.info()

In [ ]:
model = LinearRegression()
model.fit(X_final, y)

In [ ]:
y_pred = model.predict(X_final)

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

In [ ]:
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
y_prediction = model.predict(test_final)

In [ ]:
submission = pd.DataFrame({"Id": test_final["Id"], "SalePrice": y_prediction})
submission.to_csv("submission.csv", index=False)